# Coordinate systems and transforms of Synthesis AI face dataset

At first, we install some dependencies.

Installing `face_api_dataset` from the github it it's not already installed.

In [1]:
%%bash

(python -c "import face_api_dataset")
if [ $? -eq 0 ]
then
    echo "face_api_dataset is already installed"
else
    echo "installing face_api_dataset from github"
    pip install git+https://github.com/Synthesis-AI-Dev/face_api_dataset.git
fi

face_api_dataset is already installed


Installing `git lfs` from the github it it's not already installed.

In [2]:
%%bash

(git lfs install > /dev/null)
if [ $? -eq 0 ]
then
    echo "git lfs is already installed"
else
    echo "installing git lfs"
    case "$(uname -s)" in

       Darwin)
         brew install git-lfs
         ;;

       Linux)
         curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
         apt-get install git-lfs
         ;;

       *)
         echo "This cell can't install git lfs your OS. Please take care of it yourself."
         ;;
    esac
fi

git lfs is already installed


Pulling the test dataset for the example

In [3]:
%%bash

if [ -d "../test_dataset" ]
then
    echo "test_dataset is already present"
else
    echo "downloading test_dataset"
    git clone https://github.com/Synthesis-AI-Dev/face_api_dataset
    mv face_api_dataset/test_dataset ..
    rm -rf face_api_dataset
fi

test_dataset is already present


Finally we add some imports for the visualisation.

In [4]:
%matplotlib inline

In [5]:
import numpy as np

We use `FaceApiDataset` class to access synthesis datasets.


In [6]:
from face_api_dataset import FaceApiDataset, Modality

**Warning!** Some of modalities requires additional libraries to be installed:
`SEGMENTS` and `RGB` modalities use `opencv-python` library,
 while `DEPTH`, `ALPHA` and `NORMALS` modalities
 use `tiffile`  and `imagecodecs` libraries for effective work with floating point tiff files.
 If dataset with these modalities will be created without corresponding libraries present, an `ImportError` is raised.


In [7]:
data_root = "../test_dataset"
dataset = FaceApiDataset(data_root,
                        modalities=[Modality.RGB, Modality.CAM_TO_HEAD, Modality.HEAD_TO_CAM,
                                    Modality.WORLD_TO_HEAD, Modality.HEAD_TO_WORLD, Modality.WORLD_TO_CAM,
                                    Modality.CAM_TO_WORLD
                                   ])

In [8]:
len(dataset)

13

There are 13 items in the test dataset. Let's explore them closer.

In [9]:
item = dataset[0]
item2 = dataset[1]

Each item is a dict with different modalities as keys.

In [10]:
print(item.keys())

dict_keys([<Modality.RGB: 2>, <Modality.HEAD_TO_CAM: 27>, <Modality.CAM_TO_HEAD: 28>, <Modality.HEAD_TO_WORLD: 29>, <Modality.WORLD_TO_HEAD: 30>, <Modality.CAM_TO_WORLD: 31>, <Modality.WORLD_TO_CAM: 32>])


In Synthesis AI face dataset we use 3 different coordinate systems:
1.  World coordinate system.
2.  Camera coordinate system.
3.  Face coordinate system.

We provide transformations between head,camera and world coordinates systems as 4x4 matrices in homogeneous coordinates.

In [11]:
item[Modality.CAM_TO_HEAD]

array([[ 0.93239178,  0.02019352,  0.36088381,  0.38612463],
       [ 0.06352884,  0.97373945, -0.21862133, -0.23481784],
       [-0.35582242,  0.22676728,  0.90662398,  0.91507426],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [12]:
np.testing.assert_allclose(item[Modality.CAM_TO_HEAD] @ item[Modality.HEAD_TO_CAM], np.eye(4), atol=1e-10)


In [13]:
item[Modality.HEAD_TO_WORLD]

array([[ 0.93239212,  0.06352874, -0.3558217 , -0.01929043],
       [ 0.0201936 ,  0.97373974,  0.22676736,  1.55267489],
       [ 0.36088476, -0.21862136,  0.9066242 ,  0.04655426],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [14]:
np.testing.assert_allclose(item[Modality.HEAD_TO_WORLD] @ item[Modality.WORLD_TO_HEAD], np.eye(4), atol=1e-10)

In [15]:
item[Modality.CAM_TO_WORLD]

array([[ 1.00000000e+00, -1.16382334e-18,  1.40946303e-18,
         2.08167286e-04],
       [ 1.16382334e-18,  1.00000000e+00,  6.58739521e-17,
         1.53932965e+00],
       [-1.40946303e-18, -6.58739521e-17,  1.00000000e+00,
         1.06686541e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [16]:
np.testing.assert_allclose(item[Modality.CAM_TO_WORLD] @ item[Modality.WORLD_TO_CAM], np.eye(4), atol=1e-10)



In [18]:
np.testing.assert_allclose(item[Modality.CAM_TO_WORLD], item[Modality.HEAD_TO_WORLD] @ item[Modality.CAM_TO_HEAD], atol=1e-10)

